In [1]:
from preprocess_income_data import preprocess_data

X_train, X_test, X_val, y_train, y_val, y_test = preprocess_data()

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC
import numpy as np
from const import N_SPLITS, RANDOM_STATE, SVM, SVM_ID

ALGORITHM_NAME = SVM
ALGORITHM_ID = SVM_ID

kf = KFold(n_splits=20, random_state=RANDOM_STATE, shuffle=True)
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

gamma = ["auto", "scale"]

steps = [(ALGORITHM_NAME, SVC(
    random_state=RANDOM_STATE,
    ))]


pipeline = Pipeline(steps)

parameters = {
    f"{ALGORITHM_NAME}__C": C,
    f"{ALGORITHM_NAME}__gamma": gamma,
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=kf, scoring="accuracy")

cv.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=20, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('support_vector_machines',
                                        SVC(random_state=42))]),
             param_grid={'Support Vector Machines__C': [0.001, 0.01, 0.1, 1, 10,
                                                        100, 1000],
                         'Support Vector Machines__gamma': ['auto', 'scale']},
             scoring='accuracy')

In [3]:
cv.best_score_

0.8563616039444769

In [4]:
from process_cv_results import process_cv_scores, process_best_params, get_classification_metrics, results_to_csv
accuracy_scores_df = process_cv_scores(cv, algorithm_id=ALGORITHM_ID)
accuracy_scores_df.head()


,algorithm_id,accuracy_score
0,3,0.755064
1,3,0.755064
2,3,0.789441
3,3,0.837937
4,3,0.850829


In [5]:
best_params_df = process_best_params(
    cv, algorithm_id=ALGORITHM_ID, algorithm_name=ALGORITHM_NAME)
best_params_df.head()


,algorithm_id,hyperparameter_name,best_value
0,3,C,1
1,3,gamma,scale


In [6]:
y_pred = cv.predict(X_test)
classification_metrics_df = get_classification_metrics(
    y_test, y_pred, cv, ALGORITHM_ID, ALGORITHM_NAME)

classification_metrics_df.head()


[[5875  369]
 [ 797 1099]]

---


,algorithm_id,algorithm_name,best_accuracy,precision,recall,roc_auc_score,true_positives,false_positives,false_negatives,true_negatives
0,3,support_vector_machines,0.856362,0.748638,0.579641,0.760272,5875,369,797,1099


In [7]:
dfs = [accuracy_scores_df, best_params_df, classification_metrics_df]

results_to_csv(dfs, ALGORITHM_ID)
